In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import TargetEncoder
from sklearn.preprocessing import LabelEncoder
from joblib import dump

import sys 
import os
import json
sys.path.append("../src")
from data_preparation import DataPreparation

# Import des données

In [2]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

/var/folders/12/33kq22951gn7jfpz3d32v_qc0000gn/T/ipykernel_32725/176984250.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("../data/train.csv")
/var/folders/12/33kq22951gn7jfpz3d32v_qc0000gn/T/ipykernel_32725/176984250.py:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv("../data/test.csv")


# Préparation des données

In [3]:
preprocess = DataPreparation(train, test)
train, test = preprocess.prepare_data()

Valeurs manquantes du train supprimées ✅
Valeurs manquantes du test supprimées ✅
Variables renommées ✅
Valeurs manquantes numériques imputées ✅
Valeurs manquantes catégorielles imputées ✅


In [4]:
train.to_csv("../data/train_clean.csv")
test.to_csv("../data/test_clean.csv")

In [2]:
train = pd.read_csv("../data/train_clean.csv")
test = pd.read_csv("../data/test_clean.csv")

In [6]:
train[train.select_dtypes(exclude="object").columns.to_list()].head()

,Unnamed: 0,ID,m_(kg),Mt,Ewltp_(g/km),W_(mm),At1_(mm),At2_(mm),ec_(cm3),ep_(KW),Fuel_consumption_,Electric_range_(km)
0,0,0,1387.0,1534.0,401.053306,2700.0,1571.0,1576.0,999.0,92.0,5.6,0.0
1,1,1,1172.0,1337.0,394.684459,2552.0,1500.0,1483.0,999.0,70.0,5.5,0.0
2,2,2,1204.0,1335.0,398.561951,2552.0,1500.0,1483.0,999.0,70.0,5.6,0.0
3,3,3,1438.0,1588.0,479.470056,2650.0,1555.0,1563.0,1591.0,150.0,6.8,0.0
4,4,4,1207.0,1350.0,421.854820,2552.0,1500.0,1483.0,999.0,81.0,5.9,0.0


In [6]:
train.loc[train['Ewltp_(g/km)'] <= 0, 'Ewltp_(g/km)'] = 0

In [7]:
var_explicatives = ['VFN', 'Tan', 'T', 'Va', 'Ve', 'Cn', 'Ft', 'Fm', 'Fuel_consumption_', 'Electric_range_(km)']

In [8]:
X = train[var_explicatives]
test_rf = test[var_explicatives]
y = train["Ewltp_(g/km)"]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [12]:
encoder = TargetEncoder()
cat_col = train[var_explicatives].select_dtypes(include="object")
for col in cat_col:
    X_train[col] = encoder.fit_transform(X_train[col].values.reshape(-1,1),y_train)
    X_test[col] = encoder.transform(X_test[col].values.reshape(-1,1))
    test_rf[col] = encoder.transform(test_rf[col].values.reshape(-1,1))

/var/folders/12/33kq22951gn7jfpz3d32v_qc0000gn/T/ipykernel_32725/2816646882.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_rf[col] = encoder.transform(test_rf[col].values.reshape(-1,1))
/var/folders/12/33kq22951gn7jfpz3d32v_qc0000gn/T/ipykernel_32725/2816646882.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_rf[col] = encoder.transform(test_rf[col].values.reshape(-1,1))
/var/folders/12/33kq22951gn7jfpz3d32v_qc0000gn/T/ipykernel_32725/2816646882.py:6: SettingWithCopyWarning: 
A value is tr

# Modélisation Random Forest

In [13]:
N_ESTIMATORS = 100
MAX_DEPTH = 20
N_ITERATION = 100

In [14]:
rf_model = RandomForestRegressor(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, random_state=0)

print("fit en cours... ⏳")
rf_model.fit(X_train, y_train)
print("fit terminé ✅")

fit en cours... ⏳
fit terminé ✅


In [15]:
print("predict en cours... ⏳")
y_pred = rf_model.predict(X_test)
print("predict terminé ✅")

predict en cours... ⏳
predict terminé ✅


In [16]:
mae_rf = mean_absolute_error(y_test, y_pred)
print(f"MAE de la random forest : {mae_rf}")

MAE de la random forest : 2.9940168319148577


# Sauvegarde des Artefacts

In [17]:
parameters_dictionnary = {"Model" : "RandomForest", 
                          "N_ESTIMATORS" : N_ESTIMATORS, 
                          'MAX_DEPTH' : MAX_DEPTH,
                          "N_ITERATION" : N_ITERATION,
                          "MAE" : mae_rf}

In [18]:
if not os.path.exists("../artefacts/"):
    os.makedirs("../artefacts/")

if not os.path.exists(f"../artefacts/RF_{round(mae_rf,2)}"):
    os.makedirs(f"../artefacts/RF_{round(mae_rf,2)}")

with open(f"../artefacts/RF_{round(mae_rf,2)}/parameters", "w") as json_file:
    json.dump(parameters_dictionnary, json_file)

dump(rf_model, f"../artefacts/RF_{round(mae_rf,2)}/RF.joblib")

['../artefacts/RF_2.99/RF.joblib']

# Submission

In [20]:
prediction = rf_model.predict(test_rf)

submission = test[["ID"]].copy()  
submission["Ewltp (g/km)"] = prediction
submission.to_csv("../data/sample_submission5.csv", index=False)